# Refine bounding boxes
Clean edge cases when bounding boxes overlap, cover the painting entierly, and so on.

### 0. Import libraries and load data

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import json 
ANNOTATIONS_PATH = "../../data/annotations/"

sys.path.append("../annotate_dataset/")

from ground_objects import *
from annotate_paintings_utils import *

In [ ]:
with open(f"{ANNOTATIONS_PATH}annotations_30_39.json") as f:
    annotations = json.load(f)

### 1. Define bounding box comparison functions

In [4]:
def compute_iou(box_a, box_b):
    # determine the coordinates of the intersection rectangle
    x_left = max(box_a[0], box_b[0])
    y_top = max(box_a[1], box_b[1])
    x_right = min(box_a[2], box_b[2])
    y_bottom = min(box_a[3], box_b[3])

    # compute the area of intersection rectangle
    if x_right < x_left or y_bottom < y_top:
        return 0.0  
    
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both bounding boxes
    area_a = (box_a[2] - box_a[0]) * (box_a[3] - box_a[1])
    area_b = (box_b[2] - box_b[0]) * (box_b[3] - box_b[1])

    # compute the intersection over union
    iou = intersection_area / float(area_a + area_b - intersection_area)
    return iou


In [ ]:
def is_box_inside(box_a, box_b):
    # determines if box_a is insede box_b
    return (
        box_a[0] >= box_b[0] and 
        box_a[1] >= box_b[1] and 
        box_a[2] <= box_b[2] and  
        box_a[3] <= box_b[3]  
    )

In [6]:
def compute_bbox_area_wrt_image(box, image):
    image_size = image.size[0] * image.size[1]
    box_size = (box[2] - box[0]) * (box[3] - box[1])

    return box_size / image_size

### 2. Refine all bounding boxes

In [ ]:
paintings_bboxes = {
    painting_annotation["painting_id"]: painting_annotation["bounding_boxes"]
    for painting_annotation in annotations["annotations"]
}

for painting_id, bboxes in paintings_bboxes.items():
    resized_image, image = load_image(painting_id)
    formatted_bboxes = [[bbox[0], bbox[1][0], bbox[1][1]] for bbox in bboxes]

    for index_a, box_a in enumerate(formatted_bboxes):
        for index_b, box_b in enumerate(formatted_bboxes):
            if index_a != index_b and  box_a[0] != box_b[0]:
                print([box_a[0], box_a[1]], [box_b[0], box_b[1]], compute_iou(box_a[-1], box_b[-1]))
                
    display_annotated_image(image, formatted_bboxes)